### PER + PBR 합성 전략
- PBR < 1 : 자산 대비 저평가
- PER < 10 : 수익 대비 저평가

### 맞는 데이터 가져오고 merge 하기

In [24]:
from pykrx import stock

In [25]:
year = '2023'
df_first = stock.get_market_cap_by_ticker(f'{year}0102')
df_first = df_first[['종가','시가총액']].rename(columns={'종가':'시가'})
df_first

,시가,시가총액
티커,,
005930,55500,331322931525000
373220,446000,104364000000000
207940,827000,58860898000000
000660,75700,55109779030500
051910,604000,42637775172000
...,...,...
215050,912,1934959392
288490,49,1869203000
267810,1010,1804870000


In [26]:
df_first.describe()

,시가,시가총액
count,2690.000000,2.690000e+03
mean,19262.999257,7.713959e+11
std,51201.159850,7.256943e+12
min,49.000000,1.178126e+09
25%,2636.250000,5.396002e+10
50%,6235.000000,1.092750e+11
75%,14925.000000,2.592859e+11
max,827000.000000,3.313229e+14


In [27]:
df_first = df_first.sort_values('시가총액') # 범위 나누기 정렬

In [28]:
import pandas as pd
df_first['symbol_group'] = pd.cut(df_first.reset_index().index, bins=3, labels=['소형주', '중형주', '대형주'])

In [29]:
df_first    # symbol_group 추가

,시가,시가총액,symbol_group
티커,,,
322190,132,1178126004,소형주
267060,181,1654666524,소형주
267810,1010,1804870000,소형주
288490,49,1869203000,소형주
215050,912,1934959392,소형주
...,...,...,...
051910,604000,42637775172000,대형주
000660,75700,55109779030500,대형주
207940,827000,58860898000000,대형주


### read PER + PBR from pykrx

In [30]:
df_second = stock.get_market_fundamental_by_ticker(f'{year}0102')
df_second.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
095570,8075,3.35,0.71,1707,4.72,270
006840,45961,0.00,0.35,0,1.23,200
027410,16393,6.02,0.25,684,2.67,110
282330,46849,23.63,4.31,8547,1.49,3000
138930,28745,2.69,0.22,2341,8.89,560


In [32]:
df_second = df_second[['PBR', 'PER']]

### 해당 년도 마지막 거래 종가

In [33]:
df_third = stock.get_market_ohlcv_by_ticker(f'{year}1228')
df_third = df_third[['종가']]

In [34]:
df_third

,종가
티커,
095570,4980
006840,17100
027410,3810
282330,131300
138930,7140
...,...
079980,4120
005010,4985
000540,3070


### merge

In [36]:
df_concat = pd.concat([df_first, df_second, df_third], axis=1)
df_concat

,시가,시가총액,symbol_group,PBR,PER,종가
티커,,,,,,
322190,132.0,1.178126e+09,소형주,NaN,NaN,NaN
267060,181.0,1.654667e+09,소형주,NaN,NaN,NaN
267810,1010.0,1.804870e+09,소형주,NaN,NaN,NaN
288490,49.0,1.869203e+09,소형주,NaN,NaN,NaN
215050,912.0,1.934959e+09,소형주,NaN,NaN,NaN
...,...,...,...,...,...,...
45014K,NaN,NaN,NaN,NaN,NaN,5810.0
452260,NaN,NaN,NaN,NaN,NaN,1358.0
45226K,NaN,NaN,NaN,NaN,NaN,4365.0


In [37]:
df_concat_dropna = df_concat.dropna()
df_concat_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 002787 to 005930
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   시가            905 non-null    float64 
 1   시가총액          905 non-null    float64 
 2   symbol_group  905 non-null    category
 3   PBR           905 non-null    float64 
 4   PER           905 non-null    float64 
 5   종가            905 non-null    float64 
dtypes: category(1), float64(5)
memory usage: 43.4+ KB


In [38]:
df_concat_dropna['수익률'] = df_concat_dropna['종가'] / df_concat_dropna['시가']
df_concat_dropna

/tmp/ipykernel_63435/3453670861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat_dropna['수익률'] = df_concat_dropna['종가'] / df_concat_dropna['시가']


,시가,시가총액,symbol_group,PBR,PER,종가,수익률
티커,,,,,,,
002787,12800.0,3.773542e+09,소형주,0.00,0.00,12480.0,0.975000
001527,12900.0,3.980540e+09,소형주,0.00,0.00,12490.0,0.968217
001525,6530.0,4.033783e+09,소형주,0.00,0.00,6010.0,0.920368
002785,4850.0,4.157507e+09,소형주,0.00,0.00,4670.0,0.962887
000545,6140.0,4.715520e+09,소형주,0.00,0.00,5650.0,0.920195
...,...,...,...,...,...,...,...
051910,604000.0,4.263778e+13,대형주,2.17,12.82,499000.0,0.826159
000660,75700.0,5.510978e+13,대형주,0.84,5.41,141500.0,1.869221
207940,827000.0,5.886090e+13,대형주,11.09,140.67,760000.0,0.918984


In [43]:
conditions = (df_concat_dropna['PER'] >= 2.5) & (df_concat_dropna['PER'] <= 10)
#df_concat_dropna[conditions]
df_concat_dropna[conditions].sort_values('PBR').head(5)

,시가,시가총액,symbol_group,PBR,PER,종가,수익률
티커,,,,,,,
012630,5230.0,3.124492e+11,대형주,0.12,2.53,6890.0,1.317400
003830,117000.0,1.553760e+11,중형주,0.18,5.39,105700.0,0.903419
085620,2915.0,5.160022e+11,대형주,0.20,4.83,4765.0,1.634648
005720,4080.0,2.184594e+11,대형주,0.21,7.83,4165.0,1.020833
058650,104000.0,4.160000e+11,대형주,0.21,8.33,108000.0,1.038462


티커
002787    False
001527    False
001525    False
002785    False
000545    False
          ...  
051910    False
000660     True
207940    False
373220    False
005930     True
Name: PER, Length: 905, dtype: bool